# LLM Nl2Sql Flow using RAG (large database use case)

1. Azure AI Search
2. AzureOpenAI Embeddings (text-embeddings-ada-002)
3. Azure OpenAI LLM (GPT-4-32k)
4. Langchain


In [ ]:
# pip install wget
# pip install azure-search-documents 
# pip install azure-identity
# pip install openai

**Step 1: Import required libraries**


In [8]:
import os
import json
import openai  
import pandas as pd
from openai import AzureOpenAI
from langchain_openai import AzureChatOpenAI
from azure.core.credentials import AzureKeyCredential  
from azure.search.documents import SearchClient  
from azure.search.documents.indexes import SearchIndexClient  
from azure.search.documents.models import VectorizedQuery
from azure.search.documents.indexes.models import (
    HnswAlgorithmConfiguration,
    HnswParameters,
    SearchField,
    SearchableField,
    SearchFieldDataType,
    SearchIndex,
    SemanticConfiguration,
    SemanticField,
    SemanticPrioritizedFields,
    SemanticSearch,
    SimpleField,
    VectorSearch,
    VectorSearchAlgorithmKind,
    VectorSearchAlgorithmMetric,
    VectorSearchProfile,
)
import sys
from tenacity import retry, wait_random_exponential, stop_after_attempt
from langchain_core.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate, ChatPromptTemplate, PromptTemplate
from pandas import DataFrame as pd
from langchain_core.output_parsers import StrOutputParser
from langchain_community.utilities.sql_database import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool


**Step 2: Configure OpenAI settings**


In [9]:
#Langchain configuration variables
LANGCHAIN_TRACING_V2 = "true"
LANGCHAIN_API_KEY = os.environ["LANGCHAIN_API_KEY"]

azure_openai_api_endpoint=os.getenv("Az_OPENAI_ENDPOINT") 
azure_openai_api_version=os.getenv("Az_OPENAI_VERSION") 
azure_openai_api_key = os.getenv("Az_OPENAI_KEY") 
#azure_openai_api_type="azure"
azure_openai_deployment_model=os.getenv("Az_OPENAI_DEPLOYMENT_NAME")
azure_openai_embedding_model ="text-embedding-ada-002"#os.getenv("Az_OPENAI_EMB_DEPLOYMENT_NAME")

# Use API key authentication
client = AzureOpenAI(
        api_key=azure_openai_api_key,
        api_version=azure_openai_api_version,
        azure_endpoint=azure_openai_api_endpoint,
    )


**Step 3: Configure Azure AI Search Vector Store settings**


In [10]:
search_service_endpoint = os.getenv("Az_SEARCH_ENDPOINT")
search_service_api_key = os.getenv("Az_SEARCH_KEY")
table_index_name = "nl2sql-table-index"
column_index_name = "nl2sql-columns-index"

credential = AzureKeyCredential(search_service_api_key)

**Step 4: Create index for tables**


**_4.0 Function to generate embeddings, also used for query embeddings_**


In [11]:
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text):
    response = client.embeddings.create(input=text, model=azure_openai_embedding_model)
    embedding = response.data[0].embedding
    return embedding

**_Step 4.1 Tables Index Creation_**


In [12]:
# Initialize the SearchIndexClient
index_client = SearchIndexClient(
    endpoint=search_service_endpoint, credential=credential
)

# Define the fields for the index
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="dataset_name", type=SearchFieldDataType.String),
    SearchableField(name="table_name", type=SearchFieldDataType.String),
    SearchableField(name="description", type=SearchFieldDataType.String),
    SearchField(
        name="example_queries", 
        type=SearchFieldDataType.Collection(SearchFieldDataType.String),
    ),
    SearchField(
        name="description_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        vector_search_dimensions=1536,
        vector_search_profile_name="my-vector-config",
    ),
]

# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="my-hnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE,
            ),
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="my-vector-config",
            algorithm_configuration_name="my-hnsw",
        )
    ],
)

# Configure the semantic search configuration
semantic_search = SemanticSearch(
    configurations=[
        SemanticConfiguration(
            name="my-semantic-config",
            prioritized_fields=SemanticPrioritizedFields(
                title_field=SemanticField(field_name="dataset_name"),
                keywords_fields=[SemanticField(field_name="table_name")],
                content_fields=[SemanticField(field_name="description")],
            ),
        )
    ]
)

# Create the search index with the vector search and semantic search configurations
index = SearchIndex(
    name=table_index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
)

# Create or update the index
result = index_client.create_or_update_index(index)
print(f"{result.name} created")

nl2sql-table-index created


**_Step 4.2: Load tables metadata and generate embeddings_**


In [13]:

# Read the text-sample.json
with open("./tables.json", "r", encoding="utf-8") as file:
    input_data = json.load(file)

i = 0
# Generate embeddings for dataset, table and description fields
for item in input_data:
    item["id"] = str(i)
    dataset = item["dataset_name"]
    table = item["table_name"]
    description = item["description"]
    examples = item["example_queries"]

    description_embeddings = generate_embeddings(description)
    item["description_Vector"] = description_embeddings
    
    i=i+1
# Output embeddings to docVectors.json file
with open("tableVectors.json", "w") as f:
    json.dump(input_data, f)

**_Step 4.3: Upload table documents to AI Search_**


In [14]:
# Upload  embedding documents to the index
with open("tableVectors.json", "r") as file:
    documents = json.load(file)
search_client = SearchClient(
    endpoint=search_service_endpoint, index_name=table_index_name, credential=credential
)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents")

Uploaded 34 documents


**Step 5: Create index for columns**


**_Step 5.1: Column Index Creation_**


In [15]:
# Initialize the SearchIndexClient
index_client = SearchIndexClient(
    endpoint=search_service_endpoint, credential=credential
)

# Define the fields for the index
fields = [
    SimpleField(name="id", type=SearchFieldDataType.String, key=True),
    SearchableField(name="dataset_name", type=SearchFieldDataType.String),
    SearchableField(name="table_name", type=SearchFieldDataType.String),
    SearchableField(name="column_name", type=SearchFieldDataType.String),
    SearchableField(name="description", type=SearchFieldDataType.String),
    SearchableField(name="usage", type=SearchFieldDataType.String),
    SearchableField(name="data_type", type=SearchFieldDataType.String),
    SearchField(
        name="description_vector",
        type=SearchFieldDataType.Collection(SearchFieldDataType.Single),
        vector_search_dimensions=1536,
        vector_search_profile_name="my-vector-config",
    ),
]

# Configure the vector search configuration
vector_search = VectorSearch(
    algorithms=[
        HnswAlgorithmConfiguration(
            name="myHnsw",
            kind=VectorSearchAlgorithmKind.HNSW,
            parameters=HnswParameters(
                m=4,
                ef_construction=400,
                ef_search=500,
                metric=VectorSearchAlgorithmMetric.COSINE,
            ),
        )
    ],
    profiles=[
        VectorSearchProfile(
            name="my-vector-config",
            algorithm_configuration_name="myHnsw",
        )
    ],
)

# Configure the semantic search configuration
semantic_search = SemanticSearch(
    configurations=[
        SemanticConfiguration(
            name="my-semantic-config",
            prioritized_fields=SemanticPrioritizedFields(
                title_field=SemanticField(field_name="dataset_name"),
                keywords_fields=[SemanticField(field_name="table_name"),SemanticField(field_name="column_name")],
                content_fields=[SemanticField(field_name="description")],
            ),
        )
    ]
)

# Create the search index with the vector search and semantic search configurations
index = SearchIndex(
    name=column_index_name,
    fields=fields,
    vector_search=vector_search,
    semantic_search=semantic_search,
)

# Create or update the index
result = index_client.create_or_update_index(index)
print(f"{result.name} created")

nl2sql-columns-index created


**_5.2: Load data and generate document embeddings_**


In [16]:
# Read the text-sample.json
with open("./columns.json", "r", encoding="utf-8") as file:
    input_data = json.load(file)

i = 0
# Generate embeddings for dataset, table and description fields
for item in input_data:
    item["id"] = str(i)
    dataset = item["dataset_name"]
    table = item["table_name"]
    column = item["column_name"]
    description = item["description"]
    usage = item["usage"]
    datatype = item["data_type"]

    description_embeddings = generate_embeddings(description)
    item["description_Vector"] = description_embeddings

    i=i+1
# Output embeddings to docVectors.json file
with open("columnVectors.json", "w") as f:
    json.dump(input_data, f)

**_Step 5.3: upload documents to AI Search_**


In [17]:
# Upload  embedding documents to the index
with open("columnVectors.json", "r") as file:
    documents = json.load(file)
search_client = SearchClient(
    endpoint=search_service_endpoint, index_name=column_index_name, credential=credential
)
result = search_client.upload_documents(documents)
print(f"Uploaded {len(documents)} documents")

Uploaded 67 documents


**Step 6: Perform a vector similarity search on tables**


In [18]:
# Pure Vector Search for tables
query = "Provide a list of all flight reservations from October 10th to November 15th, 2023"
  
search_client = SearchClient(search_service_endpoint, table_index_name, credential)  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=5, fields="description_vector")
  
matched_documents = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query], 
    select=["dataset_name", "table_name", "description", "example_queries"],
    search_mode="all"
)

matched_tables = []
for document in matched_documents:
    dataset_name = document['dataset_name']
    table_name = document['table_name']
    matched_tables.append(f'{dataset_name}.{table_name}')
  
print(matched_tables)

['flight_reservations.reservations', 'hotel_reservations.reservations', 'flight_reservations.flights', 'flight_reservations.transactions', 'flight_reservations.customers']


**Step 7: Perform a vector similarity search on columns**


In [19]:
# Pure Vector Search for columns
search_client = SearchClient(search_service_endpoint, column_index_name, credential)  
vector_query = VectorizedQuery(vector=generate_embeddings(query), k_nearest_neighbors=20,fields="description_vector")
  
matched_columns = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query], 
    select=["dataset_name", "table_name", "column_name", "description", "usage", "data_type"],
    search_mode="all"
)
matched_columns

matched_columns_filtered = []
for document in matched_columns:
    dataset_name = document['dataset_name']
    table_name = document['table_name']
    matched_tables.append(f'{dataset_name}.{table_name}')
    if dataset_name == 'flight_reservations': # i hard coded the table name here for now, you just need to replace it with the code that loop through matched_tables
         matched_columns_filtered.append(document)
  
print(matched_columns_filtered)



[{'description': 'A unique identifier for each flight.', 'data_type': 'INT64', 'table_name': 'reservations', 'usage': 'Used to uniquely identify and manage flight records.', 'column_name': 'flight_id', 'dataset_name': 'flight_reservations', '@search.score': 0.83983755, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, {'description': 'A unique identifier for each flight.', 'data_type': 'INT64', 'table_name': 'flights', 'usage': 'Used to uniquely identify and manage flight records.', 'column_name': 'flight_id', 'dataset_name': 'flight_reservations', '@search.score': 0.83983755, '@search.reranker_score': None, '@search.highlights': None, '@search.captions': None}, {'description': 'The arrival time of the flight.', 'data_type': 'DATETIME', 'table_name': 'flights', 'usage': 'Informs users and helps them plan their travel.', 'column_name': 'arrival_datetime', 'dataset_name': 'flight_reservations', '@search.score': 0.83931977, '@search.reranker_score': No

**Step 8: Third filter on both tables and columns top k results from search**


In [20]:
matched_columns_cleaned = []

for doc in matched_columns_filtered:
    dataset_name = doc['dataset_name']
    table_name = doc['table_name']
    column_name = doc['column_name']
    data_type = doc['data_type']
    matched_columns_cleaned.append(f'dataset_name={dataset_name}|table_name={table_name}|column_name={column_name}|data_type={data_type}')
    
matched_columns_cleaned = '\n'.join(matched_columns_cleaned)
print(matched_columns_cleaned)

dataset_name=flight_reservations|table_name=reservations|column_name=flight_id|data_type=INT64
dataset_name=flight_reservations|table_name=flights|column_name=flight_id|data_type=INT64
dataset_name=flight_reservations|table_name=flights|column_name=arrival_datetime|data_type=DATETIME
dataset_name=flight_reservations|table_name=flights|column_name=departure_datetime|data_type=DATETIME
dataset_name=flight_reservations|table_name=transactions|column_name=reservation_id|data_type=INT64
dataset_name=flight_reservations|table_name=reservations|column_name=reservation_id|data_type=INT64
dataset_name=flight_reservations|table_name=reservations|column_name=reservation_datetime|data_type=DATETIME
dataset_name=flight_reservations|table_name=flights|column_name=price|data_type=FLOAT64
dataset_name=flight_reservations|table_name=flights|column_name=destination|data_type=STRING
dataset_name=flight_reservations|table_name=flights|column_name=origin|data_type=STRING
dataset_name=flight_reservations|ta

**Step 9: Text-to-SQL generation**


In [21]:
messages = []
template = "You are a SQL master expert capable of writing complex SQL queries in Microsoft Sql Server."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
messages.append(system_message_prompt)

In [22]:
human_template = """Given the following inputs:
USER_QUERY:
--
{query}
--
MATCHED_SCHEMA: 
--
{matched_schema}
--
Please construct a SQL query using the MATCHED_SCHEMA and the USER_QUERY provided above. 

IMPORTANT: Use ONLY the column names (column_name) mentioned in MATCHED_SCHEMA. DO NOT USE any other column names outside of this. 
IMPORTANT: Associate column_name mentioned in MATCHED_SCHEMA only to the table_name specified under MATCHED_SCHEMA.
NOTE: Use SQL 'AS' statement to assign a new name temporarily to a table column or even a table wherever needed. 
"""

In [23]:
human_message = HumanMessagePromptTemplate.from_template(human_template)
messages.append(human_message)

chat_prompt = ChatPromptTemplate.from_messages(messages)

In [24]:
request = chat_prompt.format_prompt(query=query,
                                    matched_schema=matched_columns_cleaned).to_messages()
request

[SystemMessage(content='You are a SQL master expert capable of writing complex SQL queries in Microsoft Sql Server.'),
 HumanMessage(content="Given the following inputs:\nUSER_QUERY:\n--\nProvide a list of all flight reservations from October 10th to November 15th, 2023\n--\nMATCHED_SCHEMA: \n--\ndataset_name=flight_reservations|table_name=reservations|column_name=flight_id|data_type=INT64\ndataset_name=flight_reservations|table_name=flights|column_name=flight_id|data_type=INT64\ndataset_name=flight_reservations|table_name=flights|column_name=arrival_datetime|data_type=DATETIME\ndataset_name=flight_reservations|table_name=flights|column_name=departure_datetime|data_type=DATETIME\ndataset_name=flight_reservations|table_name=transactions|column_name=reservation_id|data_type=INT64\ndataset_name=flight_reservations|table_name=reservations|column_name=reservation_id|data_type=INT64\ndataset_name=flight_reservations|table_name=reservations|column_name=reservation_datetime|data_type=DATETIME\

In [25]:
# Create an instance of chat llm
llm = AzureChatOpenAI(
    azure_endpoint=azure_openai_api_endpoint,
    openai_api_version=azure_openai_api_version,
    azure_deployment=azure_openai_deployment_model,
    openai_api_key=azure_openai_api_key,
    openai_api_type="azure",
    temperature = 0
)

In [26]:
#%%time 
response = llm.invoke(request)
sql = '\n'.join(response.content.strip().split('\n')[1:-1])
print(sql)

SELECT 
    r.reservation_id,
    r.flight_id,
    r.reservation_datetime,
    r.status,
    f.departure_datetime,
    f.arrival_datetime,
    f.price,
    f.destination,
    f.origin,
    f.carrier
FROM 
    flight_reservations.reservations AS r
JOIN 
    flight_reservations.flights AS f
ON 
    r.flight_id = f.flight_id
WHERE 
    r.reservation_datetime BETWEEN '2023-10-10' AND '2023-11-15'
ORDER BY 
    r.reservation_datetime;


In [34]:
# sql = sql.replace('```sql', '')
# sql = sql.replace('```', '')
# print(sql)

SELECT 
    r.reservation_id,
    r.flight_id,
    r.reservation_datetime,
    r.status,
    f.departure_datetime,
    f.arrival_datetime,
    f.price,
    f.destination,
    f.origin,
    f.carrier
FROM 
    flight_reservations.reservations AS r
JOIN 
    flight_reservations.flights AS f
ON 
    r.flight_id = f.flight_id
WHERE 
    r.reservation_datetime BETWEEN '2023-10-10' AND '2023-11-15'




In [27]:
db_user = "sa"
db_password = "password123"
db_host = "localhost"
db_name = "flight_reservations"
db_port = 3306

db = SQLDatabase.from_uri(f"mssql+pyodbc://{db_user}:{db_password}@{db_host}/{db_name}?driver=ODBC+Driver+18+for+SQL+Server&TrustServerCertificate=yes")

In [28]:
print(db.dialect)

mssql


In [29]:
execute_query = QuerySQLDataBaseTool(db=db)
df = execute_query.invoke(sql)
df

"[(6, 6, datetime.datetime(2023, 10, 10, 10, 0), 'Confirmed', datetime.datetime(2023, 11, 25, 6, 0), datetime.datetime(2023, 11, 25, 14, 30), 550.0, 'JFK', 'SEA', 'United'), (7, 7, datetime.datetime(2023, 10, 12, 11, 30), 'Confirmed', datetime.datetime(2023, 11, 27, 20, 0), datetime.datetime(2023, 11, 27, 23, 30), 380.0, 'MIA', 'JFK', 'American'), (8, 8, datetime.datetime(2023, 10, 15, 13, 20), 'Confirmed', datetime.datetime(2023, 11, 30, 10, 0), datetime.datetime(2023, 11, 30, 13, 30), 380.0, 'JFK', 'MIA', 'American'), (9, 8, datetime.datetime(2023, 10, 20, 9, 0), 'Cancelled', datetime.datetime(2023, 11, 30, 10, 0), datetime.datetime(2023, 11, 30, 13, 30), 380.0, 'JFK', 'MIA', 'American'), (10, 8, datetime.datetime(2023, 10, 22, 15, 45), 'Confirmed', datetime.datetime(2023, 11, 30, 10, 0), datetime.datetime(2023, 11, 30, 13, 30), 380.0, 'JFK', 'MIA', 'American'), (11, 11, datetime.datetime(2023, 10, 25, 12, 30), 'Confirmed', datetime.datetime(2023, 12, 12, 10, 0), datetime.datetime(20

In [30]:
answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)
rephrase_answer = answer_prompt | llm | StrOutputParser()

In [31]:
chain = (
rephrase_answer
)

chain.invoke({"question": query, "query": sql, "result": df})

'Here is the list of all flight reservations from October 10th to November 15th, 2023:\n\n1. **Reservation ID:** 6\n   - **Flight ID:** 6\n   - **Reservation DateTime:** 2023-10-10 10:00\n   - **Status:** Confirmed\n   - **Departure DateTime:** 2023-11-25 06:00\n   - **Arrival DateTime:** 2023-11-25 14:30\n   - **Price:** $550.00\n   - **Destination:** JFK\n   - **Origin:** SEA\n   - **Carrier:** United\n\n2. **Reservation ID:** 7\n   - **Flight ID:** 7\n   - **Reservation DateTime:** 2023-10-12 11:30\n   - **Status:** Confirmed\n   - **Departure DateTime:** 2023-11-27 20:00\n   - **Arrival DateTime:** 2023-11-27 23:30\n   - **Price:** $380.00\n   - **Destination:** MIA\n   - **Origin:** JFK\n   - **Carrier:** American\n\n3. **Reservation ID:** 8\n   - **Flight ID:** 8\n   - **Reservation DateTime:** 2023-10-15 13:20\n   - **Status:** Confirmed\n   - **Departure DateTime:** 2023-11-30 10:00\n   - **Arrival DateTime:** 2023-11-30 13:30\n   - **Price:** $380.00\n   - **Destination:** JFK